In [1]:
import pandas as pd

In [2]:
training = pd.read_csv('training_dataset.csv')
testing_bodytrack = pd.read_csv('testing_bodytrack.csv')
testing_blackscholes = pd.read_csv('testing_blackscholes.csv')

In [3]:
# Make a label column. The label is "cluster active" if w_big > 1, and "cluster idle" otherwise.
training['y'] = training['w_big'].apply(lambda x: 'cluster active' if x > 1 else 'cluster idle')
testing_bodytrack['y'] = testing_bodytrack['w_big'].apply(lambda x: 'cluster active' if x > 1 else 'cluster idle')
testing_blackscholes['y'] = testing_blackscholes['w_big'].apply(lambda x: 'cluster active' if x > 1 else 'cluster idle')

training.drop('w_big', axis=1, inplace=True)
testing_bodytrack.drop('w_big', axis=1, inplace=True)
testing_blackscholes.drop('w_big', axis=1, inplace=True)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(training.drop('y', axis=1), training['y'], test_size=0.2, random_state=42)


In [4]:
from sklearn.svm import SVC

# Create a support vector classifier
clf = SVC()

# Train the classifier
clf.fit(x_train, y_train)

SVC()

In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Predict the labels of the test set: y_pred
y_pred = clf.predict(x_test)

# Compute the confusion matrix: cm
print(confusion_matrix(y_test, y_pred))

# Print the accuracy
print(accuracy_score(y_test, y_pred))

# Print the classification report
print(classification_report(y_test, y_pred))

[[350   5]
 [  0 728]]
0.9953831948291783
                precision    recall  f1-score   support

cluster active       1.00      0.99      0.99       355
  cluster idle       0.99      1.00      1.00       728

      accuracy                           1.00      1083
     macro avg       1.00      0.99      0.99      1083
  weighted avg       1.00      1.00      1.00      1083



In [6]:
# Predict the labels of the test set: y_pred
y_pred = clf.predict(testing_bodytrack.drop('y', axis=1))

# Compute the confusion matrix: cm
print(confusion_matrix(y_test, y_pred))

# Print the accuracy
print(accuracy_score(y_test, y_pred))

# Print the classification report
print(classification_report(y_test, y_pred))

ValueError: Found input variables with inconsistent numbers of samples: [1083, 1454]

In [ ]:
# Predict the labels of the test set: y_pred
y_pred = clf.predict(testing_blackscholes.drop('y', axis=1))

# Compute the confusion matrix: cm
print(confusion_matrix(y_test, y_pred))

# Print the accuracy
print(accuracy_score(y_test, y_pred))

# Print the classification report
print(classification_report(y_test, y_pred))